# Experiment Notebook: User-circulating FEI Capital Allocation Model

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to...

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
from scipy.stats import norm

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# import plotly.io as pio
# png_renderer = pio.renderers["png"]
# png_renderer.width = 1200
# png_renderer.height = 500
# # png_renderer.scale = 1

# pio.renderers.default = "png"

In [ ]:
from operator import lt, gt

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
def gen_norm_rv(n, mu, sigma):
    return norm.rvs(loc=mu, scale=sigma, size=1, random_state=n)[0]

In [ ]:
cam_deposits = [
    "fei_liquidity_pool_user_deposit",
    "fei_money_market_user_deposit",
    "fei_savings_user_deposit",
    #"fei_idle_user_deposit",
]

In [ ]:
parameter_overrides = {
#     "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
#     "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
#     "rebalancing_period": [int(365 / 4)],
#     "yield_withdrawal_period": [999],  # Disable yield-withdrawal policy
#     "yield_reinvest_period": [999],  # Disable yield-reinvestment policy
    "capital_allocation_fei_deposit_variables": [
            cam_deposits,
    ],
    "capital_allocation_rebalance_duration": [30],
    "fei_savings_rate_process": [
#         lambda _run, timestep: 0.01,
#         lambda _run, timestep: 0.01 + gen_norm_rv(timestep, 0, 0.0001),
#        lambda _run, timestep: 0.01 if timestep < 365 / 4 else (0.02 if timestep < 365 * 3/4 else 0.01),
        lambda _run, timestep: 0.01 if timestep < 365 / 4 else (0.03 if timestep < 365 * 3/4 else 0.01),

    ],
    #"money_market_utilization_rate_process": [
    #    lambda _run, timestep: 0.7, #+ gen_norm_rv(timestep, 0, 0.01),
    #]
    #"capital_allocation_exogenous_concentration": [
    #    # np.array([1, 1, 1, 1]),
    #    # np.array([1, 1, 1, 1]),
    #    # np.array([1, 1, 1, 1]),
    #]
}


In [ ]:
# Experiment configuration
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

# Analysis 1: ...

{Analysis Description}

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

## exploratory analysis of state variables

In [ ]:
def get_weight_evolution(df, subset):
    df_ = pd.DataFrame(
            df.query('subset==@subset')['capital_allocation_target_weights'].to_list(),
        )
    
    df_.index = df.query('subset==@subset')['timestamp']
    return df_

In [ ]:
get_weight_evolution(df, 0).plot(
    title='CAM Weight Evolution for Policy Setting 0',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
use_cols = [key + '_balance' for key in cam_deposits]

In [ ]:
fig = px.area(df.query('subset==0'), x='timestamp', y=use_cols, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation for Policy Setting 0",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

### money market

In [ ]:
df.query('subset==0')['fei_money_market_borrowed'].plot()

In [ ]:
df.query('subset==0')['fei_money_market_utilization'].plot()

In [ ]:
df.query('subset==0').plot(x='timestamp', y=['fei_money_market_supply_rate', 'fei_money_market_borrow_rate'])

In [ ]:
# integration: utilization rate: process -> state update where U = f(volatile asset risk/vol/price, CAM[FSD, MM, LP, IDLE])

### liquidity pool

In [ ]:
df.query('subset==0')['liquidity_pool_trading_fees'].plot()

In [ ]:
df.plot(y='fei_liquidity_pool_user_deposit_yield_rate')

In [ ]:
#df.query('subset==0')['volatile_asset_price'].plot()

### fei savings deposit

In [ ]:
df.query('subset==0')['fei_savings_rate'].plot()

In [ ]:
(df.query('subset==0')['fei_money_market_supply_rate'] - df.query('subset==0')['fei_savings_rate']).plot()

#### other

### User Circulating FEI Constituents

In [ ]:
# TODO - formatting for scientific notation
# df['your_col_here'].apply(lambda x: your_human_rounding_function(x))

In [ ]:
df.query('subset==0')[['total_user_circulating_fei'] + use_cols]

In [ ]:
def get_delta_matrix_at_timestep(df, subset, timestep):
    return pd.DataFrame(df.query('subset==@subset')['capital_allocation_rebalance_matrix'][timestep])

In [ ]:
#get_delta_matrix_at_timestep(df, 0, 100)

<b>Note:</b> Filter by appropriate subset here

### FEI deposit sizes

In [ ]:
df.plot(x='timestep', y='total_user_circulating_fei', color='subset')

In [ ]:
df.plot(
    x='timestamp',
    y='fei_liquidity_pool_user_deposit_balance',
    color='subset',
    title='User FEI Liquidity Pool Balance',
    labels={
         "fei_liquidity_pool_user_deposit_balance": "User FEI LP Balance",
     },
)

In [ ]:
df.plot(
    x='timestamp',
    y='fei_money_market_user_deposit_balance',
    color='subset',
    title='User FEI Money Market Balance',
    labels={
         "fei_money_market_user_deposit_balance": "User FEI MM Balance",
     },
)

In [ ]:
df.plot(
    x='timestep',
    y='fei_savings_user_deposit_balance',
    color='subset',
    title='User FEI Savings Deposit Balance',
    labels={
         "fei_savings_user_deposit_balance": "User FEI SD Balance",
     },
)

In [ ]:
df.plot(
    x='timestamp',
    y='fei_idle_user_deposit_balance',
    color='subset',
    title='User FEI Idle Deposit Balance',
    labels={
         "fei_idle_user_deposit_balance": "User FEI IDLE Balance",
     },
)

# other

### Descriptive stats

In [ ]:
df.plot(x='timestep', y='collateralization_ratio', color='subset')

### calc rates

In [ ]:
#[x for x in df.columns]

In [ ]:
def calc_spread(x, r):
    return (x - r).mean()
    #return ((x - r).pct_change().mean()) / (x - r).pct_change().std()

In [ ]:
def calc_spread_vol(x, r):
    return (x - r).std()
    #return ((x - r).pct_change().mean()) / (x - r).pct_change().std()

In [ ]:
def calc_SR(x, r):
    return ((x - r).pct_change().mean()) / (x - r).pct_change().std()

In [ ]:
def compute_rolling_metric(df, metric_fn, subset, window):
    df_ = df.query('subset==@subset')
    L = []
    for i in range(len(df_)):
        sr = metric_fn(
            df_['fei_money_market_supply_rate'].iloc[i:i+window],
            df_['fei_savings_rate'].iloc[i:i+window],
        )
        L.append(sr)
        
    return pd.Series(L, index=df_.index)

In [ ]:
#compute_rolling_metric(df, calc_SR, 0, 30).plot()

In [ ]:
#df['fei_savings_user_deposit_balance_pct'] = df['fei_savings_user_deposit_balance'].pct_change()
#
#df.plot(
#    x='timestep',
#    y='fei_savings_user_deposit_balance_pct',
#    color='subset',
#    title='User FEI Savings Deposit Balance',
#    labels={
#         "fei_savings_user_deposit_balance": "User FEI SD Balance",
#     },
#)